# LangChain Tools 가이드

## 개요

**도구(Tool)** 는 에이전트, 체인 또는 LLM이 외부 시스템과 상호작용하기 위한 인터페이스입니다.

도구를 활용하면 AI 모델이 단순히 텍스트를 생성하는 것을 넘어서 **실제 작업을 수행** 할 수 있게 됩니다.

### 도구 활용 사례

| 분야 | 기능 | 예시 |
|------|------|------|
| **데이터 분석** | Python 코드 실행 및 계산 수행 | 통계 분석, 데이터 시각화 |
| **정보 검색** | 웹에서 최신 정보 수집 | 실시간 뉴스, 시장 데이터 조회 |
| **웹 크롤링** | 특정 웹사이트에서 데이터 추출 | 상품 정보, 기사 내용 수집 |
| **계산 처리** | 복잡한 수학적 연산 수행 | 금융 계산, 엔지니어링 연산 |

### 학습 목표

이 튜토리얼을 통해 다음을 학습할 수 있습니다:

1. **빌트인 도구** - LangChain에서 기본 제공하는 도구 활용
2. **사용자 정의 도구** - 맞춤형 도구 제작 방법
3. **웹 검색 도구** - Tavily API를 통한 실시간 정보 검색
4. **웹 크롤링 도구** - 실제 웹사이트에서 데이터 수집
5. **도구와 LLM 연결** - AI 모델이 도구를 자동으로 활용하는 방법

### 도구 사용의 장점

기존의 AI 모델은 **학습된 데이터의 범위** 내에서만 답변이 가능했습니다. 도구를 사용하면 다음과 같은 확장된 기능을 제공할 수 있습니다:

| 장점 | 설명 |
|------|------|
| **최신 정보 접근** | 실시간 검색을 통한 최신 뉴스나 데이터 확인 |
| **정확한 계산** | 수학적 계산을 정확하게 수행 |
| **외부 시스템 연동** | 데이터베이스, API, 파일 시스템 등과 연결 |
| **작업 자동화** | 반복적인 작업을 AI가 대신 처리 |

### 목차

1. **환경 설정** - 기본 설정 및 라이브러리 로드
2. **빌트인 도구** - LangChain 기본 제공 도구
   - Python REPL 도구
   - Tavily 검색 API 도구
3. **사용자 정의 도구** - 맞춤형 도구 제작
   - @tool 데코레이터 사용법
   - 네이버 뉴스 크롤링 도구
   - 구글 뉴스 검색 도구

### 참고 자료

LangChain에 통합된 도구 리스트는 아래 링크에서 확인할 수 있습니다.

- [LangChain 통합 도구 리스트](https://python.langchain.com/docs/integrations/tools/)

In [ ]:
# API 키를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv

# API 키 정보 로드
load_dotenv(override=True)

---

# 환경 설정

도구를 사용하기 전에 필요한 환경을 설정합니다.

In [ ]:
# LangSmith 추적을 설정합니다. https://smith.langchain.com
from langchain_teddynote import logging

# 프로젝트 이름을 입력합니다.
logging.langsmith("LangGraph-Tutorial")

In [ ]:
import warnings

# 경고 메시지 무시
warnings.filterwarnings("ignore")

---

# 빌트인 도구 (Built-in Tools)

LangChain에서는 다양한 **사전 정의된 도구(Tool)** 와 **툴킷(Toolkit)** 을 제공합니다.

## Tool vs Toolkit

| 구분 | 설명 | 예시 |
|------|------|------|
| **Tool** | 특정 작업을 수행하는 **단일 도구** | 검색, 계산, 번역 |
| **Toolkit** | 관련된 여러 도구를 **묶어놓은 도구 세트** | SQL 툴킷, 파일 관리 툴킷 |

## 도구 활용의 핵심 개념

1. **플러그인 방식**: 필요에 따라 도구를 선택하여 사용
2. **표준 인터페이스**: 모든 도구는 동일한 방식으로 호출 가능
3. **체인 연결**: 여러 도구를 연결하여 복잡한 작업 수행

### 참고 자료
- [LangChain Tools/Toolkits](https://python.langchain.com/docs/integrations/tools/)

## Python REPL 도구

Python REPL(Read-Eval-Print Loop) 도구는 **실시간으로 Python 코드를 실행** 할 수 있게 해주는 도구입니다.

### REPL이란?

**REPL** 은 프로그래밍에서 중요한 개념입니다:

| 단계 | 영문 | 설명 |
|------|------|------|
| **Read** | 읽기 | 사용자가 입력한 코드를 읽기 |
| **Eval** | 평가 | 입력받은 코드를 실행하기 |
| **Print** | 출력 | 실행 결과를 출력하기 |
| **Loop** | 반복 | 이 과정을 반복하기 |

### PythonREPLTool의 특징

| 특징 | 설명 |
|------|------|
| **샌드박스 환경** | 안전한 격리된 환경에서 코드 실행 |
| **데이터 분석** | 복잡한 계산이나 데이터 처리 작업 수행 |
| **시각화** | matplotlib, seaborn 등을 활용한 그래프 생성 |
| **수학 연산** | NumPy, Pandas 등의 라이브러리 활용 |

### 주요 매개변수

- **sanitize_input**: 입력을 정제하는 옵션 (기본값: True)
- **python_repl**: PythonREPL 인스턴스 (기본값: 전역 범위에서 실행)

### 주의사항

결과를 보려면 반드시 `print(...)` 함수를 사용해야 합니다. 단순히 변수명만 입력하면 출력되지 않습니다.

In [ ]:
from langchain_experimental.tools import PythonREPLTool

# Python 코드를 실행할 수 있는 REPL 도구 생성
python_tool = PythonREPLTool()

In [ ]:
# Python 코드를 실행하고 결과를 반환
# invoke() 메서드로 도구를 실행하고 결과를 출력
print(python_tool.invoke("print(100 + 200)"))

### LLM과 Python 도구 연동하기

이제 **LLM이 자동으로 Python 코드를 작성하고 실행** 하는 시스템을 만들어보겠습니다.

#### 작동 과정

| 단계 | 설명 |
|------|------|
| 1. 요청 입력 | 사용자가 자연어로 요청 (예: "로또 번호를 생성해주세요") |
| 2. 코드 생성 | LLM이 요청을 이해하고 Python 코드 작성 |
| 3. 코드 실행 | Python 도구가 자동으로 코드를 실행 |
| 4. 결과 반환 | 실행 결과를 사용자에게 반환 |

이는 **자연어 → 코드 → 실행 → 결과** 의 자동화된 파이프라인입니다.

In [ ]:
import os
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableLambda


# Python 코드를 실행하고 중간 과정을 출력하는 함수
def print_and_execute(code, debug=True):
    """Python 코드를 실행하고 결과를 반환하는 함수"""
    if debug:
        print("<Generated Code>")
        print(code)
        print("</Generated Code>", end="\n\n")
    # python_tool을 사용하여 코드 실행
    return python_tool.invoke(code)


# LLM이 Python 코드를 작성하도록 유도하는 프롬프트 템플릿
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are an expert Python programmer, well versed in meta-programming and elegant, concise and short but well documented code. You follow the PEP8 style guide. "
            "Return only the code, no __main__, no intro, no explanation, no chatty, no markdown, no code block, no nothing. Just the code."
            "You should call generated function in the code.",
        ),
        ("human", "{input}"),
    ]
)

# OpenRouter 설정으로 ChatOpenAI 객체 생성
llm = ChatOpenAI(
    temperature=0,
    model="openai/gpt-4.1",
    api_key=os.getenv("OPENROUTER_API_KEY"),
    base_url=os.getenv("OPENROUTER_BASE_URL"),
)

# 체인 구성: 프롬프트 → LLM → 텍스트 파싱 → Python 실행
chain = prompt | llm | StrOutputParser() | RunnableLambda(print_and_execute)

In [ ]:
# 체인 실행: 자연어 요청을 Python 코드로 변환하여 실행
# LLM이 로또 번호 생성 코드를 작성하고 자동으로 실행
print(chain.invoke("로또 번호 생성기를 출력하는 코드를 작성하세요."))

## Tavily 검색 API 도구

**Tavily 검색 API** 는 AI 애플리케이션을 위해 특별히 최적화된 **실시간 웹 검색 도구** 입니다.

### Tavily의 특별한 점

일반적인 구글 검색과 달리 Tavily는 **AI가 이해하기 쉬운 형태** 로 검색 결과를 제공합니다:

| 특징 | 설명 |
|------|------|
| **구조화된 데이터** | JSON 형태로 정리된 검색 결과 |
| **정확성 최적화** | AI 답변에 적합한 신뢰할 수 있는 정보 |
| **빠른 응답** | 실시간 검색 결과 제공 |
| **안전한 콘텐츠** | 유해하거나 부적절한 내용 필터링 |

### TavilySearch 도구의 주요 기능

#### TavilySearchResults vs TavilyAnswer

| 구분 | 설명 |
|------|------|
| **TavilySearchResults** | 상세한 검색 결과 목록 반환 (URL, 내용, 메타데이터 포함) |
| **TavilyAnswer** | 검색 기반 직접 답변 제공 (요약된 답변 형태) |

### API 키 발급 및 설정

**API 키 발급 주소**: https://app.tavily.com/

발급받은 API 키를 환경변수에 설정하세요:

```bash
# .env 파일에 추가
TAVILY_API_KEY=tvly-abcdefghijklmnopqrstuvwxyz
```

### TavilySearch 주요 파라미터

| 파라미터 | 타입 | 설명 |
|----------|------|------|
| `max_results` | `int` | 반환할 검색 결과의 최대 개수 (기본값: 5) |
| `topic` | `str` | 검색 주제 또는 카테고리 ("general", "news", "finance" 등) |
| `include_answer` | `bool` | 짧은 답변 요약 포함 여부 (True: 요약 포함, False: 미포함) |
| `include_raw_content` | `bool` | 원본 HTML 콘텐츠 포함 여부 (True: HTML 원문 포함, False: 미포함) |
| `include_images` | `bool` | 관련 이미지 URL 포함 여부 (True: 이미지 포함, False: 미포함) |
| `include_image_descriptions` | `bool` | 이미지 설명 포함 여부 (True: 설명 포함, False: 미포함) |
| `search_depth` | `str` | 검색 깊이 ("basic": 빠른 검색, "advanced": 심화 검색) |
| `time_range` | `str` | 검색 기간 제한 ("day", "week", "month", "year") |
| `include_domains` | `List[str]` | 포함할 도메인 리스트 (예: ["wikipedia.org"]) |
| `exclude_domains` | `List[str]` | 제외할 도메인 리스트 (예: ["twitter.com"]) |
| `country` | `str` | 국가 코드 (예: "KR" - 한국, "US" - 미국) |
| `include_favicon` | `bool` | 파비콘(사이트 아이콘) 포함 여부 (True: 포함, False: 미포함) |

각 파라미터는 Tavily 검색 결과의 형태와 범위를 세밀하게 제어할 수 있도록 도와줍니다.

In [ ]:
from langchain_tavily import TavilySearch

# Tavily 검색 도구 생성 및 설정
tool = TavilySearch(
    max_results=1,  # 반환할 검색 결과 수 (기본값: 5)
    topic="general",  # 검색 주제(카테고리) - "general": 일반 검색, "news": 뉴스, "finance": 금융
    # include_answer=False,         # 짧은 답변 요약 포함 여부 (True: 요약 포함, False: 미포함)
    # include_raw_content=False,    # 원본 HTML 콘텐츠 포함 여부 (True: HTML 원문 포함)
    # include_images=False,         # 관련 이미지 URL 포함 여부 (True: 이미지 포함)
    # include_image_descriptions=False, # 이미지 설명 포함 여부 (True: 이미지 설명 포함)
    # search_depth="basic",         # 검색 깊이 ("basic": 빠른 검색, "advanced": 심화 검색)
    # time_range="day",             # 검색 기간 제한 ("day": 24시간, "week": 7일, "month": 30일, "year": 1년)
    # include_domains=None,         # 포함할 도메인 리스트 (예: ["wikipedia.org"])
    # exclude_domains=None,         # 제외할 도메인 리스트 (예: ["twitter.com"])
    # country=None,                 # 국가 코드 (예: "KR" - 한국, "US" - 미국)
    # include_favicon=False         # 파비콘(사이트 아이콘) 포함 여부 (True: 파비콘 포함)
)

In [ ]:
# Tavily 검색 도구 실행
# "LangChain Tools"에 대해 검색하고 결과를 반환
tool.invoke({"query": "LangChain Tools 에 대해서 알려주세요"})

---

# 사용자 정의 도구 (Custom Tool)

LangChain의 **빌트인 도구들** 도 훌륭하지만, 때로는 **특별한 요구사항** 에 맞는 맞춤형 도구가 필요할 때가 있습니다.

## 왜 사용자 정의 도구가 필요한가요?

| 필요성 | 설명 |
|--------|------|
| **비즈니스 로직** | 회사만의 특별한 계산이나 처리 방식 |
| **데이터 소스** | 내부 데이터베이스나 특수한 API 연동 |
| **특수 기능** | 기존 도구로는 해결할 수 없는 고유한 작업 |
| **성능 최적화** | 특정 작업에 최적화된 커스텀 로직 |

## @tool 데코레이터 - 함수를 도구로 변환

LangChain에서는 `@tool` 데코레이터를 사용하여 **일반 Python 함수를 도구로 쉽게 변환** 할 수 있습니다.

### 변환 과정

```python
# 1단계: 일반 Python 함수
def calculate_tax(amount: float) -> float:
    return amount * 0.1

# 2단계: @tool 데코레이터 적용
@tool
def calculate_tax(amount: float) -> float:
    """Calculate tax amount"""
    return amount * 0.1

# 3단계: LangChain 도구로 자동 변환 완료!
```

### @tool 데코레이터의 장점

| 장점 | 설명 |
|------|------|
| **간편성** | 함수 위에 한 줄만 추가하면 끝 |
| **자동 문서화** | 함수의 docstring을 도구 설명으로 자동 활용 |
| **타입 추론** | Python 타입 힌트를 통한 자동 매개변수 검증 |
| **표준 인터페이스** | 다른 LangChain 도구들과 동일한 방식으로 사용 가능 |

In [ ]:
from langchain.tools import tool


# @tool 데코레이터를 사용하여 일반 함수를 LangChain 도구로 변환
@tool
def add_numbers(a: int, b: int) -> int:
    """Add two numbers"""
    return a + b


@tool
def multiply_numbers(a: int, b: int) -> int:
    """Multiply two numbers"""
    return a * b

In [ ]:
# 덧셈 도구 실행 - 딕셔너리 형태로 매개변수 전달
add_numbers.invoke({"a": 3, "b": 4})

In [ ]:
# 곱셈 도구 실행 - 딕셔너리 형태로 매개변수 전달
multiply_numbers.invoke({"a": 3, "b": 4})

## 네이버 뉴스 크롤링 도구

실제 웹사이트에서 데이터를 수집하는 **웹 크롤링 도구** 를 만들어보겠습니다. 이 도구는 네이버 뉴스 기사 URL을 입력받아 제목과 본문 내용을 추출합니다.

### 사용되는 라이브러리

| 라이브러리 | 용도 |
|-----------|------|
| **requests** | HTTP 요청을 통해 웹페이지 데이터 가져오기 |
| **BeautifulSoup** | HTML 파싱 및 원하는 요소 추출 |
| **re** | 정규표현식을 활용한 텍스트 정리 |

In [ ]:
import re
import requests
from bs4 import BeautifulSoup


# 네이버 뉴스 크롤링 도구 - @tool 데코레이터로 LangChain 도구로 변환
@tool
def naver_news_crawl(news_url: str) -> str:
    """Crawls a 네이버 (naver.com) news article and returns the body content."""
    try:
        # 네이버 뉴스 페이지에 HTTP GET 요청 전송
        response = requests.get(news_url)

        # 요청이 성공했는지 확인
        if response.status_code == 200:
            # BeautifulSoup으로 HTML 문서 파싱
            soup = BeautifulSoup(response.text, "html.parser")

            # 뉴스 제목 추출 (네이버 뉴스의 제목 태그)
            title = soup.find("h2", id="title_area").get_text()

            # 뉴스 본문 내용 추출 (네이버 뉴스의 본문 태그)
            content = soup.find("div", id="contents").get_text()

            # 연속된 줄바꿈 문자를 하나로 정리 (텍스트 정제)
            cleaned_title = re.sub(r"\n{2,}", "\n", title)
            cleaned_content = re.sub(r"\n{2,}", "\n", content)

            return f"{cleaned_title}\n{cleaned_content}"
        else:
            return f"HTTP 요청 실패. 응답 코드: {response.status_code}"

    except Exception as e:
        return f"크롤링 오류 발생: {str(e)}"

In [ ]:
naver_news_crawl.invoke("https://n.news.naver.com/mnews/article/293/0000071509")

## 구글 뉴스 검색 도구

`langchain-teddynote` 패키지에서 제공하는 **GoogleNews** 도구를 활용하여 최신 뉴스를 검색하는 도구입니다.

### GoogleNews 도구의 특징

| 특징 | 설명 |
|------|------|
| **API 키 불필요** | RSS 피드를 사용하므로 별도 API 키가 필요하지 않습니다 |
| **실시간 검색** | news.google.com에서 제공하는 최신 뉴스 정보 |
| **키워드 검색** | 특정 키워드로 관련 뉴스 검색 가능 |
| **최신순 정렬** | 가장 최근 뉴스부터 정렬되어 제공 |

### 주요 기능

| 메서드 | 기능 |
|--------|------|
| **search_latest()** | 최신 뉴스 검색 |
| **search_by_keyword()** | 키워드 기반 뉴스 검색 |

In [ ]:
from langchain_teddynote.tools import GoogleNews

# 구글 뉴스 검색 도구 생성
news_tool = GoogleNews()

In [ ]:
# 최신 뉴스 5개 검색 및 반환
news_tool.search_latest(k=5)

In [ ]:
# "AI 투자" 키워드로 관련 뉴스 3개 검색
news_tool.search_by_keyword("AI 투자", k=3)

In [ ]:
from langchain_teddynote.tools import GoogleNews
from langchain.tools import tool
from typing import List, Dict


# 키워드 기반 뉴스 검색 도구를 LangChain 도구로 변환
@tool
def search_keyword(query: str) -> List[Dict[str, str]]:
    """Look up news by keyword"""
    # 검색어 출력 (디버깅용)
    print(query)

    # GoogleNews 인스턴스 생성
    news_tool = GoogleNews()

    # 키워드로 뉴스 검색하여 5개 결과 반환
    return news_tool.search_by_keyword(query, k=5)

In [ ]:
# 커스텀 키워드 검색 도구 실행
# "LangChain AI"에 대한 뉴스를 검색하여 반환
search_keyword.invoke({"query": "LangChain AI"})